<a href="https://colab.research.google.com/github/hakim-cyber/Comp-Science-for-physics-and-chemistry/blob/main/pw8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving AminoAcids_1864dcec12371c9246f50ea95be58790.csv to AminoAcids_1864dcec12371c9246f50ea95be58790 (2).csv
Saving ParseFormula_b55ee415d06f558ab1a91281dd7d295e.py to ParseFormula_b55ee415d06f558ab1a91281dd7d295e (2).py


{'AminoAcids_1864dcec12371c9246f50ea95be58790 (2).csv': b'Name,Three letter code,One letter code,Polarization\r\nAlanine,Ala,A,non-polar\r\nArginine,Arg,R,positive\r\nAsparagine,Asn,N,polar\r\nAspartic acid,Asp,D,negative\r\nCysteine,Cys,C,polar\r\nGlutamic acid,Glu,E,negative\r\nGlutamine,Gln,Q,polar\r\nGlycine,Gly,G,non-polar\r\nHistidine,His,H,positive\r\nIsoleucine,Ile,I,non-polar\r\nLeucine,Leu,L,non-polar\r\nLysine,Lys,K,positive\r\nMethionine,Met,M,polar\r\nPhenylalanine,Phe,F,aromatic\r\nProline,Pro,P,non-polar\r\nSerine,Ser,S,polar\r\nThreonine,Thr,T,polar\r\nTryptophan,Trp,W,aromatic\r\nTyrosine,Tyr,Y,aromatic\r\nValine,Val,V,non-polar\r\n',
 'ParseFormula_b55ee415d06f558ab1a91281dd7d295e (2).py': b'def countAtoms(string, dict={}):\r\n    curDict = copy.copy(dict)\r\n    atoms = re.findall("[A-Z]{1}[a-z]*[0-9]*", string)\r\n\r\n    for j in atoms:\r\n        atomGroups = re.match(\'([A-Z]{1}[a-z]*)([0-9]*)\', j)\r\n        atom = atomGroups.group(1)\r\n        number = atomGr

setup


In [ ]:
import os, re, time, math, urllib.parse, urllib.request
import pandas as pd
from IPython.display import display
USER_PARSER=None
try:
    import ParseFormula as PF
    for f in ("parse_formula","ParseFormula","parse","parseFormula"):
        if hasattr(PF,f): USER_PARSER=getattr(PF,f); break
except: pass
tok=re.compile(r"([A-Z][a-z]?|\(|\)|\d+)")
def pform(s):
    if USER_PARSER:
        try:
            d=USER_PARSER(s)
            return {k:int(d[k]) for k in d}
        except: pass
    t=tok.findall(s); S=[{}]; i=0
    def add(d,k,v): d[k]=d.get(k,0)+v
    while i<len(t):
        x=t[i]
        if x=="(":
            S.append({})
            i+=1
        elif x==")":
            i+=1; m=1
            if i<len(t) and t[i].isdigit(): m=int(t[i]); i+=1
            g=S.pop()
            for k,v in g.items(): add(S[-1],k,m*v)
        elif x.isdigit():
            i+=1
        else:
            el=x; n=1
            if i+1<len(t) and t[i+1].isdigit(): n=int(t[i+1]); i+=1
            add(S[-1],el,n); i+=1
    return S[-1]
def pc_name(s): return urllib.parse.quote(str(s).strip().lower())
def getprop(n,prop):
    u=f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{pc_name(n)}/property/{prop}/TXT"
    with urllib.request.urlopen(u,timeout=20) as r: return r.read().decode().strip()
def getfw(n):
    try:
        f=getprop(n,"MolecularFormula")
        w=float(getprop(n,"MolecularWeight"))
        return f,w
    except:
        try:
            f=getprop("L-"+n,"MolecularFormula")
            w=float(getprop("L-"+n,"MolecularWeight"))
            return f,w
        except:
            return None, math.nan

load csv


In [ ]:
csvs=[f for f in os.listdir() if f.lower().endswith(".csv")]
CSV_PATH=csvs[0]
df=pd.read_csv(CSV_PATH)
display(df.head())

,Name,Three letter code,One letter code,Polarization
0,Alanine,Ala,A,non-polar
1,Arginine,Arg,R,positive
2,Asparagine,Asn,N,polar
3,Aspartic acid,Asp,D,negative
4,Cysteine,Cys,C,polar


 Column map

In [ ]:

df.columns = df.columns.str.strip()


rename_map = {
    "Code1": "Name",
    "Three letter code": "Code3",
    "One letter code": "Code1",
    "Polarity": "Polarity"
}

df = df.rename(columns=rename_map)

display(df.head())

,Name,Code3,Code1,Polarization
0,Alanine,Ala,A,non-polar
1,Arginine,Arg,R,positive
2,Asparagine,Asn,N,polar
3,Aspartic acid,Asp,D,negative
4,Cysteine,Cys,C,polar


enrich

In [ ]:
name_series = df["Name"] if isinstance(df["Name"], pd.Series) else df["Name"].iloc[:,0]
F=[]; W=[]
for n in name_series.astype(str):
    f,w=getfw(n); F.append(f); W.append(w); time.sleep(0.15)
df["Formula"]=F; df["MolecularWeight"]=W
df[["Name","Formula","MolecularWeight"]].head()

,Name,Formula,MolecularWeight
0,Alanine,C3H7NO2,89.09
1,Arginine,C6H14N4O2,174.20
2,Asparagine,C4H8N2O3,132.12
3,Aspartic acid,C4H7NO4,133.10
4,Cysteine,C3H7NO2S,121.16


Parse H,C,N,O,S

In [ ]:
df.columns

Index(['Name', 'Code3', 'Code1', 'Polarization', 'Formula', 'MolecularWeight'], dtype='object')

In [ ]:
H=[];C=[];N=[];O=[];S=[]
for f in df["Formula"]:
    if not isinstance(f,str) or not f: H+=[0];C+=[0];N+=[0];O+=[0];S+=[0]; continue
    d=pform(f)
    H.append(int(d.get("H",0)))
    C.append(int(d.get("C",0)))
    N.append(int(d.get("N",0)))
    O.append(int(d.get("O",0)))
    S.append(int(d.get("S",0)))
df["H"]=H; df["C"]=C; df["N"]=N; df["O"]=O; df["S"]=S
display(df.head())

,Name,Code3,Code1,Polarization,Formula,MolecularWeight,H,C,N,O,S
0,Alanine,Ala,A,non-polar,C3H7NO2,89.09,7,3,1,2,0
1,Arginine,Arg,R,positive,C6H14N4O2,174.20,14,6,4,2,0
2,Asparagine,Asn,N,polar,C4H8N2O3,132.12,8,4,2,3,0
3,Aspartic acid,Asp,D,negative,C4H7NO4,133.10,7,4,1,4,0
4,Cysteine,Cys,C,polar,C3H7NO2S,121.16,7,3,1,2,1


analyse

In [ ]:
import pandas as pd
from IPython.display import display

df = df.copy()
df.columns = [str(c).strip() for c in df.columns]
alias = {
    "polarization":"Polarity",
    "polarity":"Polarity",
    "three letter code":"Code3",
    "3-letter code":"Code3",
    "one letter code":"Code1",
    "1-letter code":"Code1",
}
for c in list(df.columns):
    k = c.strip().lower()
    if k in alias: df = df.rename(columns={c: alias[k]})
df = df.loc[:, ~df.columns.duplicated(keep="first")].copy()

if "Name" not in df: raise SystemExit("Need Name column")
if "Code3" not in df: df["Code3"] = ""
if "Code1" not in df: df["Code1"] = ""
if "Polarity" not in df: df["Polarity"] = ""
if "MolecularWeight" not in df: df["MolecularWeight"] = pd.Series([float("nan")]*len(df))
for el in ["H","C","N","O","S"]:
    if el not in df: df[el] = 0

df["MolecularWeight"] = pd.to_numeric(df["MolecularWeight"], errors="coerce")
for el in ["H","C","N","O","S"]:
    df[el] = pd.to_numeric(df[el], errors="coerce").fillna(0).astype(int)

name  = df["Name"].astype(str)
code3 = df["Code3"].astype(str)
code1 = df["Code1"].astype(str)
pol   = df["Polarity"].astype(str)

mask = name.str.lower().eq("lysine") | code3.str.upper().eq("LYS") | code1.str.upper().eq("K")
display(df[mask])

if df["MolecularWeight"].notna().any():
    display(df.loc[[df["MolecularWeight"].idxmax()]])
else:
    print("No MolecularWeight values")

p = pol.str.lower()
pos = p.str.contains("positive") | p.str.contains("basic") | p.str.fullmatch(r"\+")
display(df[pos])

display(df["Polarity"].value_counts(dropna=False).to_frame("count"))

if df["MolecularWeight"].notna().any():
    idx = df.groupby("Polarity")["MolecularWeight"].idxmax()
    display(df.loc[idx].sort_values("MolecularWeight", ascending=False))

display(df[df["S"]>=1])

display(df.loc[[df["H"].idxmax()]])

,Name,Code3,Code1,Polarity,Formula,MolecularWeight,H,C,N,O,S
11,Lysine,Lys,K,positive,C6H14N2O2,146.19,14,6,2,2,0


,Name,Code3,Code1,Polarity,Formula,MolecularWeight,H,C,N,O,S
17,Tryptophan,Trp,W,aromatic,C11H12N2O2,204.22,12,11,2,2,0


,Name,Code3,Code1,Polarity,Formula,MolecularWeight,H,C,N,O,S
1,Arginine,Arg,R,positive,C6H14N4O2,174.20,14,6,4,2,0
8,Histidine,His,H,positive,C6H9N3O2,155.15,9,6,3,2,0
11,Lysine,Lys,K,positive,C6H14N2O2,146.19,14,6,2,2,0


,count
Polarity,
non-polar,6
polar,6
positive,3
aromatic,3
negative,2


,Name,Code3,Code1,Polarity,Formula,MolecularWeight,H,C,N,O,S
17,Tryptophan,Trp,W,aromatic,C11H12N2O2,204.22,12,11,2,2,0
1,Arginine,Arg,R,positive,C6H14N4O2,174.20,14,6,4,2,0
12,Methionine,Met,M,polar,C5H11NO2S,149.21,11,5,1,2,1
5,Glutamic acid,Glu,E,negative,C5H9NO4,147.13,9,5,1,4,0
9,Isoleucine,Ile,I,non-polar,C6H13NO2,131.17,13,6,1,2,0


,Name,Code3,Code1,Polarity,Formula,MolecularWeight,H,C,N,O,S
4,Cysteine,Cys,C,polar,C3H7NO2S,121.16,7,3,1,2,1
12,Methionine,Met,M,polar,C5H11NO2S,149.21,11,5,1,2,1


,Name,Code3,Code1,Polarity,Formula,MolecularWeight,H,C,N,O,S
1,Arginine,Arg,R,positive,C6H14N4O2,174.2,14,6,4,2,0


export

In [ ]:
out="AminoAcids_enriched.csv"
df.to_csv(out,index=False)
out

'AminoAcids_enriched.csv'